In [ ]:
import numpy as np

# Symbol Parser and Equation Solver

Takes in the predicted hand-written equations/mathematical symnbols and solves the equation. 

Steps:
* Takes in a string of numbers / symbols
* Attempt to figure out the equation
* Evaluate the equation


### Attempting to parse the equation

* (TO DO)
* https://docs.sympy.org/latest/modules/parsing.html

In [ ]:
from sympy import parse_expr
from sympy import parse_expr, symbols, sqrt
from sympy.parsing.sympy_parser import transformations
from sympy.parsing.sympy_parser import standard_transformations
from sympy.parsing.sympy_parser import implicit_multiplication
from sympy.parsing.sympy_parser import function_exponentiation
print(transformations)

0: lambda_notation
1: auto_symbol
2: repeated_decimals
3: auto_number
4: factorial_notation
5: implicit_multiplication_application
6: convert_xor
7: implicit_application
8: implicit_multiplication
9: convert_equals_signs
10: function_exponentiation
11: rationalize


In [4]:
def rewrite_expression(expression):
  new_expression = []
  find_expression = False
  previous_expression = None
  symbols = "+-*/"

  for char in expression:
    # adds a multiplication if it is a digit followed by a parentheses 
    if previous_expression != None and previous_expression.isdigit() and char == "(":
      new_expression.append("*")
    
    # adds a power if it is a variable followed
    if previous_expression != None and previous_expression.isalpha() and char.isdigit():
      new_expression.append("**")
    
    # adds parentheses for sqrt (assumes the closing is before a symbol)
    if char == "sqrt":
      new_expression.append("sqrt(")
      find_expression = True
      continue
    if find_expression and char in symbols:
      new_expression.append(")")
      find_expression = False
    
    # appends character into new expression
    new_expression.append(char)
    previous_expression = char
  return new_expression

In [ ]:
expression = ["sqrt", "4", "x", "+", "1", "=", "1"]
print("Old expression: {}".format(expression))
print("New expression: {}".format(rewrite_expression(expression)))

Old expression: ['sqrt', '4', 'x', '+', '1', '=', '1']
New expression: ['sqrt(', '4', 'x', ')', '+', '1', '=', '1']


In [ ]:
expression = ['1', '+', '1', '(', '1', '-','6', ')', '/', '5'] # 1 + 1 * (1 - 6) / 5
print("Old expression: {}".format(expression))
print("New expression: {}".format(rewrite_expression(expression)))

Old expression: ['1', '+', '1', '(', '1', '-', '6', ')', '/', '5']
New expression: ['1', '+', '1', '*', '(', '1', '-', '6', ')', '/', '5']


In [ ]:
def transform_to_equation(expression):
  transformations = standard_transformations + (implicit_multiplication,) + (function_exponentiation,)
  concat_expression = "".join(expression) 
  left, right = concat_expression.split("=")
  left_expression = parse_expr(left, transformations=transformations, evaluate = False)
  right_expression = parse_expr(right, transformations=transformations, evaluate = False)
  return left_expression, right_expression

In [ ]:
  expression = ["sqrt", "4", "x", "+", "1", "=", "1"]
  expression = rewrite_expression(expression)
  transform_to_equation(expression)

(sqrt(4*x) + 1, 1)

### Simple approach to solving the expression

* Only considers evaluating numbers and symbols
* Evaluates only an expression

In [ ]:
def evaluate(expression):
    concat_expression = "".join(expression)
    try:
        return eval(concat_expression)
    except (NameError, SyntaxError, TypeError):
        print("Failed to Evaluate Expression")
        return None

In [ ]:
test = ["7", "+", "5", "*", "2"] # 7 + 5 * 2
evaluate(test)

17

### Another approach to the equation solver

* using [sympy](https://docs.sympy.org/latest/modules/solvers/solvers.html) to evaluate the equation


In [1]:
from sympy import symbols
from sympy import Eq
from sympy.solvers import solve

# this redundant to importing symbol
from sympy.abc import x, y, z

In [2]:
def evaluate2(expression):
  # evaluate a simple expression
  if not("=" in expression):
    try:
        expression = rewrite_expression(expression)
        concat_expression = "".join(expression)
        return eval(concat_expression)
    except (NameError, SyntaxError, TypeError):
        print("Failed to Evaluate Expression")
        return None
  
  # evaluate an equation
  expression = rewrite_expression(expression)
  left, right = transform_to_equation(expression)
  equation = Eq(left, right)

  return solve(equation)

In [5]:
test = ["7", "+", "5", "*", "2"] # 7 + 5 * 2
evaluate2(test)

17

In [ ]:
test2 = ["2", "*", "x", "+", "5", "=", "0"] # 2x + 5 = 0
evaluate2(test2)

[-5/2]

In [ ]:
test3 = ["sqrt", "4", "x", "+", "1", "=", "1"] # sqrt(4x) + 1 = 1
evaluate2(test3)

[0]

In [ ]:
test4 = ['1', '+', '1', '(', '1', '-', '6', ')', '/', '5'] # 1 + 1 * (1 - 6) / 5
evaluate2(test4)

0.0